## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.0,-0.0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])

## create scene builder

In [ ]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

In [ ]:
gscene = s_builder.create_gscene(crob)

In [ ]:
from pkg.utils.utils import *

In [ ]:
def read_file(filename):
    buffer = ""
    with open(filename, 'r') as f:
        while True:
            line = f.readline()
            buffer += line
            if not line: break
    return buffer

In [ ]:
dat = read_file("./data/Test6.csv")

In [ ]:
lines = dat.split("\n")
heads = lines[0].split(",")[:-1]
data_mat = []
for line in lines[1:]:
    data_line = list(map(float, line.split(",")[:-1]))
    if len(data_line)>0:
        data_mat.append(data_line)
data_mat = np.array(data_mat)

In [ ]:
data_mat

In [ ]:
JOINT_DOF = 6

In [ ]:
(heads[0:JOINT_DOF], heads[JOINT_DOF:JOINT_DOF*2], heads[JOINT_DOF*2:JOINT_DOF*3], heads[JOINT_DOF*3:JOINT_DOF*4], 
 heads[JOINT_DOF*4:JOINT_DOF*5], heads[JOINT_DOF*5:JOINT_DOF*6])

In [ ]:
idx = np.arange(len(data_mat))
# idx = np.arange(106215,106220)
q = data_mat[idx,0:JOINT_DOF]
e_dr = data_mat[idx,JOINT_DOF*2:JOINT_DOF*3]
e_nr = data_mat[idx,JOINT_DOF*3:JOINT_DOF*4]
Fext = data_mat[idx,JOINT_DOF*5:JOINT_DOF*6]
e_dn = e_dr-e_nr
q_d = q+e_dr
q_n = q+e_nr

In [ ]:
from pkg.utils.joint_utils import *

In [ ]:
Fe_list = []
for i in range(len(q)):
    Tbe = get_tf("indy0_tcp", list2dict(q[i], gscene.joint_names), gscene.urdf_content)
    Fe = np.matmul(Tbe[:3,:3].transpose(), Fext[i,:3])
    Fe_list.append(Fe)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.negative(Fe_list))
plt.grid(True)

In [ ]:
import numpy as np
np.savetxt('ForceData.csv',Fe_list,delimiter=",")